# 카멜레온 프로젝트

### 서라운드 카메라로 주변을 탐지
<img src="https://scontent-gmp1-1.xx.fbcdn.net/v/t39.30808-6/438099756_846444370849958_6209856849429948906_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=127cfc&_nc_ohc=Jmou5y-FqQoQ7kNvgGPRd9t&_nc_oc=Adhr29uLxu7_rUtrpOZMBbGrnaO2rx1k7YoPy9c-4JIy1vRTe1em-ta--NAqvT7a2y8&_nc_zt=23&_nc_ht=scontent-gmp1-1.xx&_nc_gid=AuwbsdDZ4vAcTjVPvRPm1TC&oh=00_AYDotJWN81jRlA_aG07t_7Tm6YYPu0F07uHbgGCsD67HwQ&oe=67CE5956" alt="서라운드 카메라가 탑제된 전술 방탄모" width="300"/>

### 캠의 사진을 모델에 전송
<img src="https://velog.velcdn.com/images%2Fredorangeyellowy%2Fpost%2F0716cf81-ed71-449a-a739-0d3d983d55e8%2Fimage.png" alt="영상의 전송방식" width="300"/>


### 사진의 색상 비율 분석

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans

# 🔹 YOLO 모델 로드
yolo_net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]
classes = open("coco.names").read().strip().split("\n")  # COCO 데이터셋 클래스 목록

def detect_objects(image):
    """
    이미지에서 객체 탐지 후 바운딩 박스를 반환하는 함수
    """
    height, width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    yolo_net.setInput(blob)
    detections = yolo_net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    
    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:  # 신뢰도 임계값
                box = detection[:4] * np.array([width, height, width, height])
                center_x, center_y, w, h = box.astype("int")
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    return boxes, class_ids

def extract_object_colors(image, boxes, num_colors=5):
    """
    탐지된 오브젝트 영역의 색상 비율을 분석하는 함수
    """
    all_colors = []
    all_proportions = []
    
    for (x, y, w, h) in boxes:
        roi = image[y:y+h, x:x+w]  # 바운딩 박스 영역만 추출
        pixels = roi.reshape((-1, 3))

        # KMeans 클러스터링으로 주요 색상 찾기
        kmeans = KMeans(n_clusters=num_colors, random_state=42)
        kmeans.fit(pixels)
        colors = kmeans.cluster_centers_.astype(int)

        # 색상 비율 계산
        labels = kmeans.labels_
        label_counts = Counter(labels)
        total_pixels = len(pixels)
        proportions = [count / total_pixels for count in label_counts.values()]
        
        all_colors.append(colors)
        all_proportions.append(proportions)
        
        # 🔹 파이 차트 시각화
        plt.figure(figsize=(6, 3))
        plt.pie(proportions, labels=[f'Color {i+1}' for i in range(num_colors)],
                colors=np.array(colors) / 255, autopct='%1.1f%%', startangle=90)
        plt.title("Detected Object Color Distribution")
        plt.show()
    
    return all_colors, all_proportions

# 📌 실행 코드
image_path = "your_image.jpg"  # 이미지 경로
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 1️⃣ 오브젝트 탐지
boxes, class_ids = detect_objects(image)

# 2️⃣ 탐지된 객체 주변 색상 분석
colors, proportions = extract_object_colors(image, boxes, num_colors=5)

# 3️⃣ 탐지된 오브젝트 표시
for (x, y, w, h) in boxes:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
plt.imshow(image)
plt.axis("off")
plt.show()

### 분석한 결과를 바탕으로 랜덤 패턴 생성

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter

def generate_pattern(colors, pattern_type="grid", image_size=(400, 400), block_size=50):
    """
    추출된 색상을 기반으로 랜덤 패턴을 생성하는 함수

    Parameters:
        colors (list): RGB 색상 리스트
        pattern_type (str): "grid" (격자), "stripe" (스트라이프), "dots" (랜덤 점), "mosaic" (모자이크)
        image_size (tuple): 생성할 이미지 크기 (w, h)
        block_size (int): 패턴 블록 크기 (grid, mosaic용)
    
    Returns:
        pattern_img (numpy array): 생성된 패턴 이미지
    """
    h, w = image_size
    pattern_img = np.zeros((h, w, 3), dtype=np.uint8)

    if pattern_type == "grid":
        for i in range(0, h, block_size):
            for j in range(0, w, block_size):
                color = colors[np.random.randint(len(colors))]
                pattern_img[i:i+block_size, j:j+block_size] = color

    elif pattern_type == "stripe":
        for i in range(0, w, block_size):
            color = colors[np.random.randint(len(colors))]
            pattern_img[:, i:i+block_size] = color

    elif pattern_type == "dots":
        pattern_img[:] = (255, 255, 255)  # 배경 흰색
        for _ in range(len(colors) * 10):  # 색상 개수 × 10개의 점 찍기
            x, y = np.random.randint(0, w), np.random.randint(0, h)
            radius = np.random.randint(5, 20)
            color = tuple(colors[np.random.randint(len(colors))])
            cv2.circle(pattern_img, (x, y), radius, color, -1)

    elif pattern_type == "mosaic":
        num_blocks = int((w * h) / (block_size * block_size))
        for _ in range(num_blocks):
            x, y = np.random.randint(0, w - block_size), np.random.randint(0, h - block_size)
            color = colors[np.random.randint(len(colors))]
            pattern_img[y:y+block_size, x:x+block_size] = color

    return pattern_img

# 🔹 예제 실행
sample_colors = [
    [200, 50, 50], [50, 200, 50], [50, 50, 200],  # Red, Green, Blue
    [200, 200, 50], [200, 50, 200]  # Yellow, Purple
]

pattern_types = ["grid", "stripe", "dots", "mosaic"]

plt.figure(figsize=(10, 10))
for i, pattern in enumerate(pattern_types):
    pattern_img = generate_pattern(sample_colors, pattern_type=pattern)
    plt.subplot(2, 2, i+1)
    plt.imshow(pattern_img)
    plt.axis("off")
    plt.title(pattern)

plt.show()

### 웨어러블 디스플레이 의류에 랜덤 패턴 출력
<img src="https://img1.yna.co.kr/etc/inner/KR/2015/08/05/AKR20150805048200063_01_i_P4.jpg" width="300"/>